In [1]:
!git clone https://github.com/resemble-ai/Resemblyzer.git

Cloning into 'Resemblyzer'...
remote: Enumerating objects: 608, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 608 (delta 1), reused 6 (delta 1), pack-reused 602
Receiving objects: 100% (608/608), 101.47 MiB | 599.00 KiB/s, done.
Resolving deltas: 100% (108/108), done.


In [4]:
cd Resemblyzer/

[Errno 2] No such file or directory: 'Resemblyzer/'
/home/auishik/Documents/diarization/Resemblyzer


In [20]:
import wave
import contextlib

fname = '/home/auishik/Documents/10small.wav'

with contextlib.closing(wave.open(fname,'r')) as f:
    frames = f.getnframes()
    samp_width = f.getsampwidth()
    channels = f.getnchannels()
    rate = f.getframerate()
    params = f.getparams()
    duration = frames / float(rate)
    print(params)

_wave_params(nchannels=2, sampwidth=2, framerate=48000, nframes=3497079, comptype='NONE', compname='not compressed')


In [7]:
help(contextlib)

Help on module contextlib:

NAME
    contextlib - Utilities for with-statement contexts.  See PEP 343.

MODULE REFERENCE
    https://docs.python.org/3.10/library/contextlib.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

CLASSES
    abc.ABC(builtins.object)
        AbstractAsyncContextManager
            AsyncExitStack(_BaseExitStack, AbstractAsyncContextManager)
            aclosing
        AbstractContextManager
            ExitStack(_BaseExitStack, AbstractContextManager)
            closing
            nullcontext(AbstractContextManager, AbstractAsyncContextManager)
            suppress
    builtins.object
        ContextDecorator
    _BaseExitStack(builtins.object)
        AsyncExitStack(_BaseExitStack,

In [ ]:
import pathlib
pathlib.Path().resolve()
from resemblyzer import VoiceEncoder, preprocess_wav
from pathlib import Path

fpath = Path("/home/auishik/Documents/multi speaker/10small.wav")
wav = preprocess_wav(fpath)

print(wav)


In [ ]:
encoder = VoiceEncoder('cpu')
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(wav_splits)      

In [ ]:
from spectralcluster import SpectralClusterer
from spectralcluster import AutoTune
import math
from sklearn.cluster import SpectralClustering

#clusterer = SpectralClusterer(min_clusters = 3)
# clusterer = SpectralClusterer(
#     min_clusters=2,
#     max_clusters=100,
#     p_percentile=0.95,
#     gaussian_blur_sigma=1)

clusterer = SpectralClusterer(min_clusters = 3, max_clusters = 10, custom_dist = "cosine")

#refinement_options = RefinementOptions(gaussian_blur_sigma=1,p_percentile=0.95,thresholding_soft_multiplier=0.01, thresholding_type=ThresholdType.RowMax,   refinement_sequence=ICASSP2018_REFINEMENT_SEQUENCE)
#clusterer = SpectralClusterer(min_clusters=3, max_clusters=10, custom_dist="cosine")
print(clusterer)
labels = clusterer.predict(cont_embeds)

In [ ]:
help(SpectralClusterer)

In [ ]:
print(labels[:1000])

In [ ]:
import json
def create_labelling(labels, wav_splits):
    from resemblyzer.audio import sampling_rate    
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    
    labelling = []
    start_time = 0

    for i, time in enumerate(times):
        
        if i>0 and labels[i]!=labels[i-1]:
            temp = {'id': str(labels[i-1]), 'start' : start_time, 'end' : time}
            labelling.append((temp))
            start_time = time
        if i==len(times)-1:
            temp = {'id': str(labels[i]),'start' : start_time, 'end' : time}
            labelling.append((temp))

    return (labelling) 
labelling = create_labelling(labels,wav_splits)

In [ ]:
json.dumps(labelling)

In [ ]:
type(labelling)

In [ ]:
import ffmpeg
import requests
import json
import warnings
url = "http://dev.revesoft.com:9382/"

payload={}

for i in range(len(labelling)):

    stream = ffmpeg.input('/home/auishik/Documents/multi speaker/3short.wav')
    a1 = stream.audio.filter('atrim', labelling[i][1], labelling[i][2])
    out = ffmpeg.output(a1, 'output.wav').overwrite_output()
    out.run()

    files=[('file',('output.wav',open('output.wav','rb'),'audio/wav'))]
    headers = {}
    #print(url)
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    data = response.json()
#print(data)
    #print("speaker: " + str(labelling[i][0])+": " + data['output'])
    print("speaker: "+ str(labelling[i][0])+" start time :"+str(labelling[i][1])+" end time :" + str(labelling[i][2])+" text :"+ data['output'])

print(response.text)

In [ ]:
print(type(stream))

In [ ]:
ffmpeg 

In [ ]:
import numpy as np
from resemblyzer import VoiceEncoder, preprocess_wav
from pathlib import Path

fpath = Path("/home/auishik/Documents/audio_checking_experiment/should be dual speaker/5069.wav")
wav = preprocess_wav(fpath)

encoder = VoiceEncoder()
embed = encoder.embed_utterance(wav)
np.set_printoptions(precision=3, suppress=True)
print(embed)

In [ ]:
help(preprocess_wav)